# Kepler challenge
## modelling
1. extract and select features from the preprocessed time series (based on the 37 actual + 37 augmented positives and 200 randomly drawn negatives). Positive = with an exoplanet, negative = without
2. apply PCA to the selected features
3. fit Random Forest models based on the top 20 PCs
4. perform trainsductive conformal prediction 100 times by repeating steps 1-3
5. save the modelling output

In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import ndimage

from datetime import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, settings

# set the logger to error level
# tsfresh outputs many warnings for features that cannot be calculated
import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# from tsfresh github examples
# note in this function they write normalize where they should really have written standardize!
class PCAForPandas(PCA):
    """This class is just a small wrapper around the PCA estimator of sklearn 
    including normalization to make it compatible with pandas DataFrames."""

    def __init__(self, **kwargs):
        self._z_scaler = StandardScaler()
        super(self.__class__, self).__init__(**kwargs)

        self._X_columns = None

    def fit(self, X, y=None):
        """Normalize X and call the fit method of the base class with numpy arrays
        instead of pandas data frames."""

        X = self._prepare(X)

        self._z_scaler.fit(X.values, y)
        z_data = self._z_scaler.transform(X.values, y)

        return super(self.__class__, self).fit(z_data, y)

    def fit_transform(self, X, y=None):
        """Call the fit and the transform method of this class."""

        X = self._prepare(X)

        self.fit(X, y)
        return self.transform(X, y)

    def transform(self, X, y=None):
        """Normalize X and call the transform method of the base class with numpy arrays
        instead of pandas data frames."""

        X = self._prepare(X)

        z_data = self._z_scaler.transform(X.values, y)

        transformed_ndarray = super(self.__class__, self).transform(z_data)

        pandas_df = pd.DataFrame(transformed_ndarray)
        pandas_df.columns = ["pca_{}".format(i) for i in range(len(pandas_df.columns))]

        return pandas_df

    def _prepare(self, X):
        """Check if the data is a pandas DataFrame and sorts the column names.

        :raise AttributeError: if pandas is not a DataFrame or the columns of the new X 
        is not compatible with the columns from the previous X data"""
        
        if not isinstance(X, pd.DataFrame):
            raise AttributeError("X is not a pandas DataFrame")

        X.sort_index(axis=1, inplace=True)

        if self._X_columns is not None:
            if self._X_columns != list(X.columns):
                raise AttributeError("The columns of the new X is not compatible with the columns from the previous X data")
        else:
            self._X_columns = list(X.columns)

        return X

## transductive conformal prediction (TCP)

With TCP learning and prediction occur simultaneously. In this sense confidence in a prediction is tailored both to the previously seen objects (whose features and labels are known) and to the features of the new object, whose label is to be predicted. By conditioning on the new objects conformal predictors take account of how difficult a particular object is to label and adjust their confidence in the prediction accordingly, as opposed to having an overall error rate for labelling all new objects.

In [3]:
def tcp(features_train, features_new, y_train, n_trees):
    n_labels = len(np.unique(y_train))  # number of classes
    p_values = np.zeros(n_labels)  # for storing p-values

    # Mondrian TCP
    for label in range(0, n_labels):
        model = RandomForestClassifier(n_estimators=n_trees)
        X_fit = np.append(features_train, [features_new], axis=0)
        y_fit = np.append(y_train, label)
        model.fit(X_fit, y_fit)
        samp = np.where(y_fit == label)[0]
        # non-conformity scores for objects with label i
        alpha = 1 - model.predict_proba(X_fit[samp, :])[:, label]
        # score for "test" object assuming label = i
        alpha_new = alpha[-1]
        # p-value for label i
        p_values[label] = len(np.where(alpha > alpha_new)[0])
        # more careful treatment of cases where alpha == alpha_new
        p_values[label] += np.random.uniform() * len(np.where(alpha == alpha_new)[0])
        p_values[label] /= len(alpha)

    return p_values


## load data and perform modelling
100 replicate runs based on:
1. different draws of nonexoplanet stars (200 - selected this value to not have overly imbalanced data)
2. tsfresh feature extraction and selection (selection based on independent p-value tests)
3. 20 principle componets taken as features for modelling
4. transductive conformal prediction with random forest (100 trees) as underlying algorithm

In [4]:
x_train = pd.read_csv('x_train.csv', index_col=0)
x_test = pd.read_csv('x_test.csv', index_col=0)
y_train = pd.read_csv('y_train.csv', index_col=0)
y_test = pd.read_csv('y_test.csv', index_col=0)

extraction_settings = ComprehensiveFCParameters() # for tsfresh

n_draws = 100
n_RF_trees = 100
n_neg = 200
n_test = len(y_test)
res = np.zeros((n_draws, n_test, 2))

for i in range(n_draws):
    print('draw #', i + 1, str(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    stars = y_train[y_train['LABEL']].index.values # all exoplanet stars
    ff = np.random.choice(np.arange(38, 5088), n_neg, replace=False)
    stars = np.concatenate((stars,ff))
    stars.sort()
    col_names = ['id', 'time', 'f01', 'f02']
    X0  = pd.DataFrame(columns = col_names)
    for ii in stars:
        xx = x_train[x_train['id']==ii]
        X0 = pd.concat([X0,xx]).reset_index(drop=True)
    X0['id'] = X0['id'].astype(int)
    X0['time'] = X0['time'].astype(int)
    X = extract_features(X0,
                         column_id='id', column_sort='time',
                         default_fc_parameters=extraction_settings,
                         impute_function= impute)
    Y = y_train.loc[stars]
    Y = Y.iloc[:,0] # turning into a Series insteas of a dataframe
    X_filtered = select_features(X, Y)
    
    nfeat = X_filtered.shape[1]
    ncomp = min(nfeat,20) # just incase, although mostly nfeat is >> 20
    pca_train = PCAForPandas(n_components=ncomp)
    X_train_pca = pca_train.fit_transform(X_filtered)
    # add index plus 1 to keep original index
    X_train_pca.index += 1
    
    X_test_filtered = extract_features(x_test, column_id='id', column_sort='time',
                                       kind_to_fc_parameters=settings.from_columns(X_filtered.columns),
                                       impute_function=impute)
    
    # need the following fix as there appears to be a bug in tsfresh
    # whereby too many features get extracted for the test set
    column_select = []
    xcols = X_filtered.columns.values
    for ii in range(len(xcols)):
        column_select.append(int(np.where(X_test_filtered.columns.values == xcols[ii])[0]))
        
    column_select = np.asanyarray(column_select)
    X_test_filtered = X_test_filtered.iloc[:, column_select]
    X_test_pca = pca_train.transform(X_test_filtered)
    X_test_pca.index += 1
    
    for j in range(1, n_test + 1):
        res[i, j - 1, :] = tcp(X_train_pca, X_test_pca.loc[j], Y, n_RF_trees)

draw # 1 2018-12-17 21:47:28


Feature Extraction: 100%|██████████| 20/20 [04:24<00:00,  8.02s/it]


draw # 2 2018-12-17 21:58:33


Feature Extraction: 100%|██████████| 20/20 [04:06<00:00, 12.31s/it]


draw # 3 2018-12-17 22:09:20


...

Feature Extraction: 100%|██████████| 20/20 [04:01<00:00,  6.81s/it]


draw # 99 2018-12-18 14:19:01


Feature Extraction: 100%|██████████| 20/20 [04:02<00:00,  6.74s/it]


draw # 100 2018-12-18 14:29:13


Feature Extraction: 100%|██████████| 20/20 [04:02<00:00,  6.74s/it]


## save results

In [5]:
np.save('res.npy', res)